# 🕵️‍♂️ Análisis Forense de Datos: ¿Relleno con Dummies?

**Hipótesis del Usuario:** La API de Valencia tiene ~381 sensores reales, pero la ingestión devuelve 400 registros. ¿Son los extras "dummies", vacíos o duplicados?

**Objetivos:**
1.  Cargar el último dataset.
2.  Verificar conteo total vs únicos.
3.  Buscar registros vacíos o repetidos.

In [ ]:
import os
import json
import pandas as pd
from datetime import datetime
from pathlib import Path

DATA_PATH = Path("../data/raw")

def get_latest_file(base_path):
    files = sorted(list(base_path.rglob("*.json")))
    return files[-1] if files else None

latest_file = get_latest_file(DATA_PATH)
print(f"📂 Analizando: {latest_file}")

In [ ]:
if latest_file:
    with open(latest_file, 'r') as f:
        data = json.load(f)
        
    if 'results' in data:
        df = pd.json_normalize(data['results'])
        print(f"📊 Total Registros: {len(df)}")
        
        # 1. ANÁLISIS DE DUPLICADOS (La prueba del algodón)
        # -------------------------------------------------
        # Si hay 381 reales y tenemos 400, y no son dummies, deben ser duplicados.
        # ACTUALIZADO: Usamos 'idtramo' en lugar de 'gid'
        if 'idtramo' in df.columns:
            total_ids = len(df)
            unique_ids = df['idtramo'].nunique()
            duplicados = total_ids - unique_ids
            
            print(f"🆔 IDs Únicos (idtramo): {unique_ids}")
            print(f"👯 Duplicados encontrados: {duplicados}")
            
            if duplicados > 0:
                print("\n🔍 Muestra de duplicados:")
                display(df[df['idtramo'].duplicated(keep=False)].sort_values('idtramo').head(10))
        else:
            print("⚠️ No encuentro columna 'idtramo' para chequear duplicados. Columnas:", df.columns.tolist())

        # 2. ANÁLISIS DE VACÍOS (Dummies)
        # -------------------------------
        # Buscamos filas donde el ID o el estado sean nulos
        print("\n👻 Análisis de Vacíos/Dummies:")
        print(df.isnull().sum()[df.isnull().sum() > 0])
        
        # 3. VER EL FINAL DEL DATASET
        # ---------------------------
        # A veces el relleno viene al final
        print("\n🍑 Cola del DataFrame (últimos 5):")
        display(df.tail())
        
    else:
        print("Estructura incorrecta (no 'results')")